# 使用 DataRequest.py 和 Preprocessing.py 获取并聚合金融数据

在本教程中，我们将演示如何利用 **DataRequest.py** 和 **Preprocessing.py** 两个脚本，在金融领域获取逐笔成交数据（tick data）并将其聚合为每分钟的蜡烛图数据（candlestick data）。

## 准备工作

- 确保 `DataRequest.py` 和 `Preprocessing.py` 已放置在当前工作目录中。
- 安装所需依赖库：
  ```bash
  pip install pandas finnhub-python # you should have this already
  ```
- 获取并替换您的 API 密钥，如 Finnhub 的 `YOUR_API_KEY`。

## 1. 使用 DataRequest.py 获取原始金融数据

下面分别演示如何获取 **K线数据（candle data）** 和 **逐笔成交数据（tick data）**。

In [2]:
from Utility.DataRequest import fetch_candle_data

# 示例：获取 AAPL 在 2023-01-03 的 1 分钟 K 线数据
candle_df = fetch_candle_data(
    symbol='AAPL',
    start_date='2023-01-03',
    end_date='2023-01-03',
    interval='1',        # '1' 表示 1 分钟 K 线
    token='YOUR API KEY', # API 密钥
    max_workers=1 # Number of threads to use
)

candle_df.head()

AAPL 1-min K 线块: 100%|██████████| 1/1 [00:00<00:00,  6.43it/s]


,timestamp,open,high,low,close,volume
0,2023-01-03 04:00:00-05:00,130.28,131.00,130.28,131.00,8174
1,2023-01-03 04:01:00-05:00,130.87,131.17,130.87,131.10,8820
2,2023-01-03 04:02:00-05:00,131.18,131.24,131.17,131.17,2112
3,2023-01-03 04:03:00-05:00,131.19,131.29,131.19,131.28,3888
4,2023-01-03 04:04:00-05:00,131.28,131.46,131.28,131.46,5984


### `fetch_tick_data` 参数说明

- **`page_workers`** (_int_)  
  控制在单个交易日内并行抓取“分页”数据的线程数量。  
  - tick 数据往往按“页”（page）分块返回，`page_workers` 决定同时发起多少个分页请求。  
  - 提高单日内数据抓取速度，但线程过多可能导致 API 限流，需要根据网络和 API 限制进行调优。

- **`day_workers`** (_int_)  
  控制跨多个交易日并行抓取的线程数量。  
  - 当需要获取多日 tick 数据时，`day_workers` 决定同时处理多少天的数据请求。  
  - 可在多日范围内并行拉取，提高整体抓取效率；同样需注意 API 并发限制。

In [4]:
from Utility.DataRequest import fetch_tick_data

# 示例：获取 AAPL 在 2023-01-03 的逐笔成交明细
tick_df = fetch_tick_data(
    symbol='AAPL',
    start_date='2023-01-03',
    end_date='2023-01-03',
    api_key='YOUR API KEY', # API 密钥
    page_workers= 10, # 单日内并行抓取5个分页
    day_workers= 1 # 跨天并行抓取n天数据，这里我们只抓1天，所以用1
)

tick_df.head()

Days:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching ticks for 2023-01-03 — 当日总 tick 数: 1021095


Days: 100%|██████████| 1/1 [00:08<00:00,  8.48s/it]


,timestamp,symbol,price,volume,condition
0,2023-01-03 04:00:00.004000-05:00,AAPL,130.28,100,"[1, 24]"
1,2023-01-03 04:00:00.004000-05:00,AAPL,130.28,12,"[1, 24, 12]"
2,2023-01-03 04:00:00.005000-05:00,AAPL,130.28,10,"[1, 24, 12]"
3,2023-01-03 04:00:00.007000-05:00,AAPL,130.28,4,"[1, 24, 12]"
4,2023-01-03 04:00:00.009000-05:00,AAPL,130.28,5,"[1, 24, 12]"


## 2. 使用 Preprocessing.py 将 Tick 数据聚合为每分钟蜡烛图数据

使用 `aggregate_tick_to_minute` 函数将逐笔数据聚合为 OHLCV 数据：

In [7]:
from Utility.Preprocessing import aggregate_tick_to_minute

# 将 tick 数据聚合为分钟级别 OHLCV 数据
minute_df = aggregate_tick_to_minute(tick_df)

[Info] Dropped 0 rows due to invalid timestamps.


## 3. 展示聚合结果并解释结构

查看前 5 行聚合后的分钟级数据：

In [8]:
minute_df.head(5)

,timestamp,open,high,low,close,volume,dollar_volume,vwap,tick_count,trade_size_mean,trade_size_std,zero_return_count,price_direction_ratio,large_trade_count,large_trade_ratio,large_trade_volume_ratio
0,2023-01-03 04:00:00-05:00,130.28,131.00,130.06,130.89,8174.0,1069601.58,130.854120,208.0,39.298077,89.094740,101.0,0.246377,5.0,0.024038,0.285784
1,2023-01-03 04:01:00-05:00,130.89,131.18,130.85,131.10,8820.0,1155025.24,130.955243,157.0,56.178344,188.856313,71.0,0.250000,1.0,0.006369,0.255556
2,2023-01-03 04:02:00-05:00,131.17,131.29,131.10,131.19,2112.0,277112.92,131.208769,53.0,39.849057,70.221320,23.0,0.250000,1.0,0.018868,0.210227
3,2023-01-03 04:03:00-05:00,131.17,131.29,131.15,131.28,3888.0,510186.43,131.220790,90.0,43.200000,63.307223,54.0,0.202247,2.0,0.022222,0.190329
4,2023-01-03 04:04:00-05:00,131.29,131.46,131.24,131.40,5984.0,785866.15,131.327899,88.0,68.000000,125.710888,37.0,0.264368,2.0,0.022727,0.229445


上述表格中，每一行代表一个一分钟的时间段，主要列字段说明如下：
- **timestamp**：时间戳列，表示这一分钟的开始时间（精确到分钟）。通常已经转换为美东时间（America/New_York），方便对照交易所时间。例如 `2023-01-03 04:00:00-05:00` 表示美东时间09:30这一分钟的区间。
- **open（开盘价）**：该分钟内第一笔交易的价格。如04:00那一分钟内的第一笔成交价为130.28。
- **high（最高价）**：该分钟内成交的最高价格。如04:00那一分钟内最高成交价为131.00。
- **low（最低价）**：该分钟内成交的最低价格。如04:00那一分钟内最低成交价为130.06。
- **close（收盘价）**：该分钟内最后一笔交易的价格，即分钟结束时的价格。例如 04:00 分时段的最后成交价为130.89。
- **volume（成交量）**：该分钟内所有成交的数量总和。在04:00这一分钟内，总共成交了 8174 股（例如由多笔交易累计而成）。
- **dollar_volume（美元成交额）**：该分钟内所有成交的金额总和，计算方法是每笔成交价乘以成交量再求和。例如 04:00 一分钟的成交额为 1069601.58 美元。
- **vwap（量价加权平均价）**：Volume-Weighted Average Price，按成交量加权的平均价格。上例中04:00的 VWAP 大约为130.85，表示成交主要集中在靠近130.85美元的价格水平。
- **tick_count（交易笔数）**：该分钟内发生的成交笔数。例如04:00有208笔成交。
- **trade_size_mean（平均单笔成交量）**：该分钟内每笔交易的平均成交量。例如04:00分钟内平均每笔成交量为39.29股。
- **trade_size_std（成交量标准差）**：该分钟内单笔成交量的标准差，用于衡量交易笔大小的波动性。04:00这一分钟内成交量标准差约为89.09。
- **zero_return_count（零回报次数）**：该分钟内连续成交价**未发生变动**的次数。换句话说，有多少次成交价与前一笔相同。这对于了解价格停滞或持平的频率很有用。表中示例值为0表示每分钟内价格每次变动都不同（没有重复价格连续出现）。
- **price_direction_ratio（价格上行比例）**：在该分钟内，价格变动中**上涨**的比例，即 `(涨价次数) / (总变动次数)`。如果价格一直在涨，这个值趋近于1；一直跌则趋近于0；0.5则表示涨跌次数相当。示例中各分钟该值为0.5，表示价格上涨和下跌次数持平。
- **large_trade_count（大单交易笔数）**：该分钟内大笔交易（成交量异常大的交易）的次数。判断大单的标准通常是基于成交量的统计分布，例如超过平均值多个标准差（代码中采用了大约 2 个标准差作为阈值）。示例中为0表示这一分钟没有异常大的单笔交易。
- **large_trade_ratio（大单笔数占比）**：大单交易笔数占该分钟总交易笔数的比例。如果有大单发生，这个比率表示大单在笔数上占的比例。例子中为0表示无大单，所以占比0%。
- **large_trade_volume_ratio（大单成交量占比）**：该分钟内大单成交的数量占总成交量的比例。用于衡量成交量中有多少由超大笔交易贡献。示例中为0表示总成交量中没有来自异常大单的部分。

综上，通过 `Preprocessing.py` 的聚合，我们将原始高频的 tick 数据转换成了易于分析的每分钟蜡烛图数据表 `minute_df`。  
这张表结构清晰：前几列是经典的 OHLCV 数据，可用于绘制价格走势图表；后面的列则提供了额外的市场微观结构信息，供深入分析交易行为之用。  
对于初学者而言，可以先关注 OHLC 和成交量等基本字段；对于高级分析，可以进一步利用 VWAP 和微观结构特征来研究市场细节。

**至此，我们已经完成了整个流程**：从使用 `DataRequest.py` 获取金融市场的原始数据，到使用 `Preprocessing.py` 将逐笔数据聚合为每分钟级别的数据表，并理解了结果的结构和含义。您现在可以将这些步骤应用到其他股票或时间范围的分析中，并根据需要调整参数（例如符号、日期范围、K线粒度等）来获取所需的数据。祝您学习愉快！